In [43]:
# Initial Imports
import os
from dotenv import load_dotenv

import pandas as pd
import numpy as np
from datetime import datetime

import finnhub
import ta #Technical Analysis library

load_dotenv()

True

In [2]:
# Setting up Finnhub API
finnhub_api_key = os.getenv("FINNHUB_API_KEY")
finnhub_sandbox_key = os.getenv("FINNHUB_SANDBOX_KEY")
finnhub_client = finnhub.Client(api_key = finnhub_sandbox_key)

In [3]:
def getUNIX(date):
    """
    Input date in YYYY-MM-DD format (as a string) and returns the associated UNIX timestamp
    """
    # Parsing the input date
    dateparts = date.split("-")
    year = int(dateparts[0])
    month = int(dateparts[1])
    day = int(dateparts[2])
    
    unix = int((datetime(year, month, day) - datetime(1970,1,1)).total_seconds())
    return unix

In [4]:
def getYMD(unix):
    """
    Input a UNIX timestamp and returns a date in the format of YYYYMMDD
    Any additional hours, minutes, or seconds are dropped
    """
    ts = int(unix)
    return datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d')

In [5]:
def getOHLCV(ticker, startDate, endDate):
    """
    Input a ticker, startDate (YYYY-MM-DD), endDate (YYYY-MM-DD)
    Returns daily open, high, low, close, and volume (in that order) in a pandas dataframe for the given criteria
    """
    # @TODO: Error handling -- check to see that endDate is after startDate, endDate has already passed, etc.
    
    # Converting to UNIX timestamp
    startDate = getUNIX(startDate)
    endDate = getUNIX(endDate)
    
    # Calling Finnhub API for candles data
    candlesData = finnhub_client.stock_candles(ticker, 'D', startDate, endDate)
    OHLCV = pd.DataFrame(candlesData)
    
    # Dropping the column denoting status of response and any null fields
    OHLCV.drop(columns = "s", inplace = True)
    OHLCV.dropna(inplace = True)
    
    # Renaming columns for ease of interpretation
    OHLCV = OHLCV.rename(columns = {
        "c":"close",
        "h":"high",
        "l":"low",
        "o":"open",
        "t":"date",
        "v":"volume"
    })
    
    # Converting UNIX timestamp to date and setting date as index
    OHLCV["date"] = OHLCV["date"].apply(getYMD)
    OHLCV.set_index(OHLCV["date"], inplace = True)
    OHLCV.drop(columns = "date", inplace = True)
    
    # Reordering columns to match OHLCV
    OHLCV = OHLCV[["open", "high", "low", "close", "volume"]]
    
    return OHLCV

In [53]:
def getTechIndicators(OHLCV):
    """
    Input OHLCV dataframe. Please note that due to the windows of the SMAs, at least 200 day's worth of data will be required. 
    Rows with null values will be automatically dropped
    
    Calculates daily returns and the following technical indicators:
    
    Simple Moving Average if Closing prices with 20, 50, and 100 day windows
    Exponential Moving Average of Daily Returns Volatility with 20, 50, and 100 day windows
    Moving Average Convergence Divergence (MACD)

    On Balance Volume (OBV)
    Chaikin Money Flow (CMF)
    
    Awesome Oscillator (AOsc)
    
    Relative Strength Index (RSI)
    Stochastic Oscillator (SOsc)
    """
    # @TODO: Validate OHLCV has at least 200 rows
    
    # Calculating Daily Returns
    OHLCV["DR"] = OHLCV["close"].pct_change()
    
    # Calculating the Simple Moving Averages of Closing Prices
    OHLCV["SMA20"] = OHLCV["close"].rolling(window = 20).mean()
    OHLCV["SMA50"] = OHLCV["close"].rolling(window = 50).mean()
    OHLCV["SMA100"] = OHLCV["close"].rolling(window = 100).mean()
    
    # Calculating EWM of Daily Return Volatility
    OHLCV["EWM20V"] = OHLCV["DR"].ewm(halflife = 20).std()
    OHLCV["EWM50V"] = OHLCV["DR"].ewm(halflife = 50).std()
    OHLCV["EWM100V"] = OHLCV["DR"].ewm(halflife = 100).std()
    
    # Calculating MACD
    ewm26 = OHLCV['close'].ewm(halflife = 26).mean()
    ewm12 = OHLCV['close'].ewm(halflife = 12).mean()
    OHLCV["MACD"] = ewm12 - ewm26
    
    OHLCV["MACDEWM9"] = OHLCV["MACD"].rolling(window = 9).mean()
    
    # Calculating on balance volume
    OHLCV["OBV"] = ta.volume.on_balance_volume(OHLCV["close"], OHLCV["volume"])
    
    # Calculating Chaikin Money Flow
    OHLCV["CMF"] = ta.volume.ChaikinMoneyFlowIndicator(
        high = OHLCV["high"], 
        low = OHLCV["low"], 
        close = OHLCV["close"], 
        volume = OHLCV["volume"]).chaikin_money_flow()
    
    # Calculating Awesome Oscillator
    OHLCV["AOsc"] = ta.momentum.AwesomeOscillatorIndicator(
        high = OHLCV["high"], 
        low = OHLCV["low"]).awesome_oscillator()
    
    # Calculating RSI
    OHLCV["RSI"] = ta.momentum.RSIIndicator(close = OHLCV["close"]).rsi()
    
    # Calculating Stochastic Oscillator
    OHLCV["SOsc"] = ta.momentum.StochasticOscillator(
        high = OHLCV["high"], 
        low = OHLCV["low"], 
        close = OHLCV["close"]).stoch()
    
    # Dropping rows with null values
    OHLCV.dropna(inplace = True)
    
    return OHLCV

In [ ]:
# @TODO: Create signal interpretations (bear/bull/neutral)
# Based on SMAs
# Based on Volatility
# Based on Oscillators


In [64]:
def indAnalysis(TechInd):
    """
    Gives Bullish (1.0), Bearish (-1.0), or Neutral (0.0) outlook based on provided technical indicators.
    
    TechInd dataframe should contain the following indicators:
        - Simple Moving Average if Closing prices with 20, 50, and 100 day windows
        - Exponential Moving Average of Daily Returns Volatility with 20, 50, and 100 day windows
        - Moving Average Convergence Divergence (MACD)
        On Balance Volume (OBV)
        - Chaikin Money Flow (CMF)
        Awesome Oscillator (AOsc)
        Relative Strength Index (RSI)
        Stochastic Oscillator (SOsc)
    """
    TechA = TechInd.iloc[:,0:5]
    
    # Signal for SMA crossovers
    SMA2050Bull = np.where(TechInd["SMA20"] > TechInd["SMA50"], 1.0, 0.0)
    SMA2050Bear = np.where(TechInd["SMA20"] < TechInd["SMA50"], -1.0, 0.0)
    TechA["SMA20-50"] = SMA2050Bull + SMA2050Bear
    
    SMA50100Bull = np.where(TechInd["SMA50"] > TechInd["SMA100"], 1.0, 0.0)
    SMA50100Bear = np.where(TechInd["SMA50"] < TechInd["SMA100"], -1.0, 0.0)
    TechA["SMA50-100"] = SMA50100Bull + SMA50100Bear
    
    # Signal for EWM Volatility crossovers
    EWM2050VBull = np.where(TechInd["EWM20V"] < TechInd["EWM50V"], 1.0, 0.0)
    EWM2050VBear = np.where(TechInd["EWM20V"] > TechInd["EWM50V"], -1.0, 0.0)
    TechA["EWM20-50V"] = EWM2050VBull + EWM2050VBear
    
    EWM50100VBull = np.where(TechInd["EWM50V"] < TechInd["EWM100V"], 1.0, 0.0)
    EWM50100VBear = np.where(TechInd["EWM50V"] > TechInd["EWM100V"], -1.0, 0.0)
    TechA["EWM50-100V"] = EWM50100VBull + EWM50100VBear
    
    # Signal for MACD crossover with MACD signal line
    MACDBull = np.where(TechInd["MACD"] > TechInd["MACDEWM9"], 1.0, 0.0)
    MACDBear = np.where(TechInd["MACD"] < TechInd["MACDEWM9"], -1.0, 0.0)
    TechA["MACDCross"] = MACDBear + MACDBull
    
    # Signal for CMF
    CMFBull = np.where(TechInd["CMF"] > 0, 1.0, 0.0)
    CMFBear = np.where(TechInd["CMF"] < 0, -1.0, 0.0)
    TechA["CMFSig"] = CMFBull + CMFBear
    
    # Signal for AOsc
    AOscBull = np.where(TechInd["AOsc"] > 0, 1.0, 0.0)
    AOscBear = np.where(TechInd["AOsc"] < 0, -1.0, 0.0)
    TechA["AOscSig"] = AOscBull + AOscBear
    
    return TechA

In [65]:
OHLCV = getOHLCV("AAPL", "2019-01-20", "2020-01-20")
TechInd = getTechIndicators(OHLCV)
IndAna = indAnalysis(TechInd)

In [67]:
IndAna.tail(50)

,open,high,low,close,volume,SMA20-50,SMA50-100,EWM20-50V,EWM50-100V,MACDCross,CMFSig,AOscSig
date,,,,,,,,,,,,
2019-11-06,160.475006,160.925007,159.600000,160.774994,189661240,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2019-11-07,161.725006,162.724991,161.324997,162.150002,237350840,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2019-11-08,161.674995,162.775002,160.524998,162.574997,175204960,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2019-11-11,161.450005,164.050007,161.424999,163.875008,205074600,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2019-11-12,163.474998,164.249992,163.075008,163.724995,218472260,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2019-11-13,163.199997,165.475006,163.174992,165.300007,258175920,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2019-11-14,164.850006,165.550003,163.824997,164.150009,223955560,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2019-11-15,164.799995,166.100006,164.375000,166.100006,250936660,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2019-11-18,166.124992,167.150002,165.149994,166.949997,217008960,1.0,1.0,1.0,1.0,1.0,1.0,1.0
